<div class="alert alert-info">
<a href='http://116.85.5.40:8778/notebooks/erlangai/onnx/tf_trt.ipynb'>运行本Notebook, 请使用8778端口</a>
</div>

In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -p numpy,sklearn,pandas
%watermark -p cv2,PIL,matplotlib
%watermark -p torch,torchvision,torchaudio
%watermark -p tensorflow,tensorboard
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Image, Javascript
from IPython.core.magic import register_line_cell_magic, register_line_magic, register_cell_magic
display(HTML('<style>.container { width:%d%% !important; }</style>' % 95))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp
import numpy as np

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            for s in ['main', 'master']:
                uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            for s in ['/raw/main/', '/raw/master/']:
                uri = 'https://' + '/'.join(mod[:3]) + s + '/'.join(mod[3:]) + '.py'
                x = requests.get(uri)
                if x.status_code == 200:
                    x = x.text
                    break
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass

def _DIR_(x):
    return json.dumps([y for y in dir(x) if not y.startswith('_')])

numpy 1.19.5
sklearn not installed
pandas 1.1.5
cv2 4.5.3
PIL 8.3.1
matplotlib 3.3.4
torch not installed
torchvision not installed
torchaudio not installed
tensorflow 2.6.0
tensorboard 2.6.0


In [2]:
def display_html(port, height=600):
    from IPython import display
    from html import escape as html_escape
    frame_id = 'erlangai-frame-{:08x}'.format(random.getrandbits(64))
    shell = '''
      <iframe id='%HTML_ID%' width='100%' height='%HEIGHT%' frameborder='0'>
      </iframe>
      <script>
        (function() {
          const frame = document.getElementById(%JSON_ID%);
          const url = new URL(%URL%, window.location);
          const port = %PORT%;
          if (port) {
            url.port = port;
          }
          frame.src = url;
        })();
      </script>
    '''
    replacements = [
        ('%HTML_ID%', html_escape(frame_id, quote=True)),
        ('%JSON_ID%', json.dumps(frame_id)),
        ('%HEIGHT%', '%d' % height),
        ('%PORT%', '%d' % port),
        ('%URL%', json.dumps('/')),
    ]
    for (k, v) in replacements:
        shell = shell.replace(k, v)
    display.display(display.HTML(shell))
    
@register_line_magic
def tensorboard(line):
    if line.strip() == 'help':
        print('%tensorboard logdir port [height]')
        return
    import signal, shlex
    from tensorboard import manager as tbmanager

    args = line.split()
    logdir, port, height = args[0], int(args[1]), int(args[2]) if len(args) == 3 else 600
    
    infos = tbmanager.get_all()
    for info in infos:
        if info.port != port: continue
        try:
            os.kill(info.pid, signal.SIGKILL)
            os.unlink(os.path.join(tbmanager._get_info_dir(), f'pid-{info.pid}.info'))
        except OSError as e:
            if e.errno != errno.ENOENT: raise
        except Exception:
            pass
        break

    strargs = f'--host 0.0.0.0 --port {port} --logdir {logdir} --reload_interval 10'
    command = shlex.split(strargs, comments=True, posix=True)
    tbmanager.start(command)
    display_html(port, height)

@register_line_magic
def netron(line):
    if line.strip() == 'help':
        print('%tensorboard file port [height]')
        return
    args = line.split()
    file, port, height = args[0], int(args[1]), int(args[2]) if len(args) == 3 else 600
    # res = !lsof -i:$port | grep $port
    # if len(res) == 1:
    #     pid = int(res[0].split(' ')[1])
    #     !kill -9 $pid
    import netron
    try:
        netron.start(file, address=('0.0.0.0', port), browse=False)
    except:
        pass
    display_html(port, height)


## 准备工作

In [ ]:
!nvidia-smi

In [4]:
import shutil
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np
import pathlib

DATA_ROOT = '/data/nb_data/tf_trt'
SAVED_MODEL_DIR = f'{DATA_ROOT}/saved_models'

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    print(device)
    tf.config.experimental.set_memory_growth(device, True)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


### 模型格式

<center>

格式|简介
:---|:---
Checkpoint|用于保存模型的权重，主要用于模型训练过程中参数的备份和模型训练热启动
GraphDef|不包含模型权重，加上checkpoint后就有模型上线的全部信息
SavedModel|包含模型Graph和权限可直接用于上线，TensorFlow和Keras模型推荐使用这种模型格式
FrozenGraph|使用freeze_graph.py对checkpoint和GraphDef进行整合和优化，可以直接部署到Android、iOS等移动设备上
TFLite|基于flatbuf对模型进行优化，可以直接部署到Android、iOS等移动设备上，使用接口和FrozenGraph有些差异

</center>

## 模型转换

In [6]:
# resnet50_weights_tf_dim_ordering_tf_kernels.h5
model = tf.keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [ ]:
type(model)

In [8]:
_DIR_(model)

'["activity_regularizer", "add_loss", "add_metric", "add_update", "add_variable", "add_weight", "apply", "build", "built", "call", "compile", "compiled_loss", "compiled_metrics", "compute_dtype", "compute_mask", "compute_output_shape", "compute_output_signature", "count_params", "distribute_strategy", "dtype", "dtype_policy", "dynamic", "evaluate", "evaluate_generator", "finalize_state", "fit", "fit_generator", "from_config", "get_config", "get_input_at", "get_input_mask_at", "get_input_shape_at", "get_layer", "get_losses_for", "get_output_at", "get_output_mask_at", "get_output_shape_at", "get_updates_for", "get_weights", "history", "inbound_nodes", "input", "input_mask", "input_names", "input_shape", "input_spec", "inputs", "layers", "load_weights", "losses", "make_predict_function", "make_test_function", "make_train_function", "metrics", "metrics_names", "name", "name_scope", "non_trainable_variables", "non_trainable_weights", "optimizer", "outbound_nodes", "output", "output_mask", "

### Model Summary

In [9]:
len(model.layers), type(model.input), type(model.output), type(model.layers[0]), type(model.layers[-1])

(177,
 keras.engine.keras_tensor.KerasTensor,
 keras.engine.keras_tensor.KerasTensor,
 keras.engine.input_layer.InputLayer,
 keras.layers.core.Dense)

In [10]:
model.input, model.output, model.inputs, model.outputs

(<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>,
 [<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>],
 [<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>])

In [11]:
model.layers[0].get_input_at(0), model.layers[0].get_output_at(0), \
model.layers[-1].get_input_at(0), model.layers[-1].get_output_at(0)

(<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>,
 <KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'avg_pool')>,
 <KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>)

In [12]:
input_node_names = [model.layers[0].get_output_at(0).name.split(':')[0]]
output_node_names = [model.layers[-1].get_output_at(0).name.split(':')[0]]
input_node_names, output_node_names 

(['input_1'], ['predictions/Softmax'])

In [13]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

### Keras Frozen

In [14]:
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

In [15]:
type(full_model), type(frozen_func), type(frozen_func.graph)

In [16]:
_DIR_(frozen_func), '-'*80, _DIR_(frozen_func.graph)

In [17]:
frozen_func.inputs, frozen_func.outputs

In [18]:
for i, op in enumerate(frozen_func.graph.get_operations()[:10]):
    print('%2d' % i, op.name)

In [22]:
frozen_graph_proto_path = tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir=DATA_ROOT,
    name="resnet50_frozen.pb",
    as_text=False)

In [ ]:
# %netron {frozen_graph_proto_path} 9001

### Convert to ONNX

In [ ]:
inputs = ','.join([x.name for x in frozen_func.inputs])
outputs = ','.join([x.name for x in frozen_func.outputs])
inputs, outputs

In [ ]:
!python3 -m tf2onnx.convert --opset 14 --input {frozen_graph_proto_path} \
    --inputs {inputs} --outputs {outputs} --output {frozen_graph_proto_path}.onnx

In [ ]:
%netron {frozen_graph_proto_path}.onnx 9002

### ONNX to TensorRT

In [21]:
%%writefile {DATA_ROOT}/install_tensorrt.sh

sudo apt-get install libnvinfer8 libnvonnxparsers8 libnvparsers8 libnvinfer-plugin8
sudo apt-get install python3-libnvinfer

Overwriting /data/nb_data/tf_trt/install_tensorrt.sh


In [23]:
trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH

<NetworkDefinitionCreationFlag.EXPLICIT_BATCH: 0>

In [46]:
# %%writefile {DATA_ROOT}/onnx2tensorrt.py

import tensorrt as trt

def onnx2tensorrt(onnx_file_path, engine_file_path):
    G_LOGGER = trt.Logger(trt.Logger.WARNING)
    EXPLICIT_BATCH = 1
    with trt.Builder(G_LOGGER) as builder, builder.create_network(EXPLICIT_BATCH) as network, \
            trt.OnnxParser(network, G_LOGGER) as parser:
        config = builder.create_builder_config()
        config.max_workspace_size = 1 << 20
        config.set_flag(trt.BuilderFlag.FP16)
        config.set_flag(trt.BuilderFlag.STRICT_TYPES)

        builder.max_batch_size = 1

        print('Loading ONNX file from path {}...'.format(onnx_file_path))
        with open(onnx_file_path, 'rb') as model:
            print('Beginning ONNX file parsing')
            if not parser.parse(model.read()):
                for e in range(parser.num_errors):
                    print(parser.get_error(e))
                raise TypeError("Parser parse failed.")

        print('Completed parsing of ONNX file')

        print('Building an engine from file {}; this may take a while...'.format(onnx_file_path))
        engine = builder.build_engine(network, config)
        print(type(engine))
        print("Created engine success! ")

        print('Saving TRT engine file to path {}...'.format(engine_file_path))
        with open(engine_file_path, "wb") as f:
            f.write(engine.serialize())
        print('Engine file has already saved to {}!'.format(engine_file_path))
        return engine

In [32]:
_DIR_(trt.tensorrt.Builder)

'["build_engine", "build_serialized_network", "create_builder_config", "create_network", "create_optimization_profile", "error_recorder", "gpu_allocator", "is_network_supported", "max_DLA_batch_size", "max_batch_size", "num_DLA_cores", "platform_has_fast_fp16", "platform_has_fast_int8", "platform_has_tf32"]'

In [47]:
onnx2tensorrt(f'{frozen_graph_proto_path}.onnx', f'{frozen_graph_proto_path}.plan')

Loading ONNX file from path /data/nb_data/tf_trt/resnet50_frozen.pb.onnx...
Beginning ONNX file parsing
Completed parsing of ONNX file
Building an engine from file /data/nb_data/tf_trt/resnet50_frozen.pb.onnx; this may take a while...
<class 'NoneType'>
Created engine success! 
Saving TRT engine file to path /data/nb_data/tf_trt/resnet50_frozen.pb.plan...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Use build_serialized_network instead.


AttributeError: 'NoneType' object has no attribute 'serialize'

## AA

# BB

## 模型训练保存

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(f'{DATA_ROOT}/mnist.npz')

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the model architecture
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(
  train_images,
  train_labels,
  epochs=1,
  validation_data=(test_images, test_labels)
)

model.save(SAVED_MODEL_DIR)

In [ ]:
train_images[0].shape

In [ ]:
model.inputs, model.outputs, model.outputs[0].name

In [ ]:
model.summary()

## 查看签名信息(signatures)

In [ ]:
!saved_model_cli show --dir {SAVED_MODEL_DIR} --tag_set serve --signature_def serving_default

### 冻结计算图(frozen graph)

In [ ]:
# from tensorflow.python.tools import freeze_graph
# 
# freeze_graph.freeze_graph(
#     input_saved_model_dir=SAVED_MODEL_DIR,
#     output_graph=f'{SAVED_MODEL_DIR}/freeze_graph.pb',
#     saved_model_tags = 'serve',
#     output_node_names='dense/BiasAdd',
#     initializer_nodes='',
#     input_graph=None,
#     input_saver=False,
#     input_binary=False,
#     input_checkpoint=None,
#     restore_op_name=None,
#     filename_tensor_name=None,
#     clear_devices=True,
#     input_meta_graph=False,
# ) 

## 计算图可视化

In [ ]:
from tensorflow.python.client import session
from tensorflow.python.framework import importer
from tensorflow.python.framework import ops
from tensorflow.python.summary import summary
from tensorflow.python.tools import saved_model_utils

model_dir = SAVED_MODEL_DIR
log_dir = f'{DATA_ROOT}/logs'
tag_set = 'serve'

shutil.rmtree(log_dir)

T = None

with session.Session(graph=ops.Graph()) as sess:
    input_graph_def = saved_model_utils.get_meta_graph_def(model_dir, tag_set).graph_def

    importer.import_graph_def(input_graph_def)

    pb_visual_writer = summary.FileWriter(log_dir)
    pb_visual_writer.add_graph(sess.graph)
    T = sess.graph
    pb_visual_writer.flush()
    pb_visual_writer.close()
    

In [ ]:
%tensorboard {log_dir} 8901 800

In [ ]:
%netron {model_dir}/keras_metadata.pb 8902

## Optimizer

### Tensorrt

    sudo apt-get install -y --no-install-recommends libnvinfer6=6.0.1-1+cuda10.1 \
        libnvinfer-dev=6.0.1-1+cuda10.1 \
        libnvinfer-plugin6=6.0.1-1+cuda10.1

In [ ]:
# from tensorflow.python.compiler.tensorrt import trt_convert as trt
# converter = trt.TrtGraphConverterV2(input_saved_model_dir=SAVED_MODEL_DIR)
# converter.convert()
# converter.save(SAVED_MODEL_DIR)

# References

- [This guide provides instructions for installing TensorFlow for Jetson Platform.][13]
- [TensorRT Install][12]
- [TensorRT (TF-TRT) optimization][8]
- [使用 TensorFlow、ONNX 和 TensorRT 加速深度学习推理][9]
- [How to export a TensorFlow 2.x Keras model to a frozen and optimized graph][10]
- [Pytorch to TensorRT][11]
- [利用 TensorRT 实现 TensorFlow 低延迟推理][1]
- [如何查看Tensorflow SavedModel格式模型的信息][2]
- [What is difference frozen_inference_graph.pb and saved_model.pb?][3]
- [浅谈tensorflow模型保存为pb的各种姿势][4]
- [github: import_pb_to_tensorboard.py][5]
- [github: freez_grpah][6]
- [tensorflow 模型导出总结][7]

[13]: https://docs.nvidia.com/deeplearning/frameworks/install-tf-jetson-platform/index.html
[12]: https://docs.nvidia.com/deeplearning/tensorrt/install-guide/index.html
[11]: https://github.com/qq995431104/Pytorch2TensorRT/
[10]: https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb
[9]: https://developer.nvidia.com/zh-cn/blog/speeding-up-deep-learning-inference-using-tensorflow-onnx-and-tensorrt/
[8]: https://docs.nvidia.com/deeplearning/frameworks/tf-trt-user-guide/index.html
[7]: https://zhuanlan.zhihu.com/p/113734249
[6]:https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph.py
[5]: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/import_pb_to_tensorboard.py
[4]: https://cloud.tencent.com/developer/article/1737892
[3]: https://newbedev.com/what-is-difference-frozen-inference-graph-pb-and-saved-model-pb
[2]: https://cloud.tencent.com/developer/article/1454661
[1]: https://discuss.tf.wiki/t/topic/1391

# FAQs

- [错误 Could not load dynamic library libnvinfer.so.6 解决方法][1]
- [cuda.h not found when pip install pycuda][2]

[2]: https://blog.csdn.net/lyyiangang/article/details/118794393
[1]: https://blog.csdn.net/zywvvd/article/details/106863908

In [ ]:
np.random.randn()